In [1]:
import numpy as np

In [2]:
interval = [-5.12, 5.12]
n = 10
l = 10
m = 100
# interval = [2, 5]

In [3]:
def rastrigin(X, A = 10):
    
    """ Rastrigin function. """
    
    n = len(X)
    Y = A*n + sum([np.square(x) - A*np.cos(2*np.pi*x) for x in X])
    
    return Y

In [4]:
def decoding(B, l, interval):
    
    """ Decoding each binary solution to a floating point representation. """
    
    n = int(len(B)/l)
    X = np.zeros((n,))
    
    # The interval between adjacent values
    delta = (interval[1] - interval[0])/(np.power(2,l)-1)  
    
    # For each variable b in B
    for i in range(n):
        b = B[l*i:l*(i+1)]
        X[i] = interval[0] + delta * sum([b[k]*np.power(2,(l-k-1)) for k in range(l)])

    return X

In [5]:
def encoding(X, l, interval):
    
    """ Encoding each real solution to a binary representation. """
    
    n = len(X)
    B = list()
    
    # For each variable X[i]
    for i in range(n):        
        k = int(np.ceil((X[i] - interval[0])*(np.power(2,l)-1)/(interval[1] - interval[0])))
        b = bin(k).replace("0b", "")
        b = [float(bi) for bi in b]
        
        if len(b) < l:
            b = list(np.zeros((l-len(b),))) + b
            
        B.append(b)    
    
    B = np.array(B).reshape((-1))
    
    return B

In [6]:
X = np.zeros((n,))
#X = np.array([-5.12]*l)
print(X)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
B = encoding(X, l = l, interval = interval)
print(B)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [8]:
X = decoding(B, l = l, interval = interval)
print(X)

[0.00500489 0.00500489 0.00500489 0.00500489 0.00500489 0.00500489
 0.00500489 0.00500489 0.00500489 0.00500489]


In [9]:
rastrigin(X)

0.04969096075727464

In [10]:
def variable_length_crossover(parent_a, parent_b, l, n, cross_prob):
    
    """ One crossover per variable. """
    
    assert (len(parent_a) == len(parent_b)), "Parents with different lengths"

    prob = float(np.random.uniform(low = 0, high = 1))

    if prob < cross_prob:        
        
        # List of crossover points
        points = list()
        
        # Offspring
        offspring_a = np.array(list())
        offspring_b = np.array(list())
        
        # For each variable b in B
        for i in range(n):
            
            # Crossover point
            point = int(np.random.randint(low = ((l*i)+1), high = (l*(i+1)-1), size = 1))
            points.append(point)
            
            # First offspring
            head = parent_a[(l*i):point]
            tail = parent_b[point:(l*(i+1))]
            offspring_a = np.concatenate([offspring_a, head, tail], axis = 0)
            
            # Second offspring
            head = parent_b[(l*i):point]
            tail = parent_a[point:(l*(i+1))]
            offspring_b = np.concatenate([offspring_b, head, tail], axis = 0)
            
    else:
        
        # Offspring will be a copy of their parents
        offspring_a = parent_a.copy()
        offspring_b = parent_b.copy()
        
    print(points)
        
    return offspring_a, offspring_b

In [11]:
parent_a = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
parent_b = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
print(parent_a)
print(parent_b)

[0 1 0 1 0 1 0 1 0 1]
[1 0 1 0 1 0 1 0 1 0]


In [12]:
offspring_a, offspring_b = variable_length_crossover(parent_a, parent_b, l = l, n = n, cross_prob = 1)
print(offspring_a.reshape((1,10)))
print()
print(offspring_b.reshape((1,10)))

[3, 15, 22, 38, 47, 53, 62, 71, 84, 91]
[[0. 1. 0. 0. 1. 0. 1. 0. 1. 0.]]

[[1. 0. 1. 1. 0. 1. 0. 1. 0. 1.]]


In [13]:
def mutation(parent, l, n, mut_prob):
    
    """ Bit-flip mutation. Changes each gene (0 to 1 or 1 to 0) with a probability mut_prob. """
    
    for i in range(n*l):
        
        prob = float(np.random.uniform(low = 0, high = 1))
        
        if prob < mut_prob:
            print(i)
            parent[i] = int(not(parent[i]))
            
    return parent

In [14]:
def initial_population(l, n, m, interval):
    
    """ Random population initialization. """
    
    # Empty population
    population = list()
    fitness = np.zeros((m,))
        
    # Generating initial population
    for i in range((m)):
        X = np.random.uniform(low = interval[0], high = interval[1], size = n)
        fitness[i] = rastrigin(X)
        B = encoding(X, l, interval)
        population.append(B)
        
    population = np.array(population).reshape((m,(n*l)))
    
    return population, fitness

In [15]:
population, fitness = initial_population(l, n, m, interval)

In [16]:
def roulette_wheel(population, m, fitness):
    
    """ Fitness proportionate selection. """
    
    # Segment sizes
    a = np.ones((m+1,))
    
    # Building the Roulette Wheel
    for i in range(m):
        # Minimization problem
        a[i+1] = 1 - sum(fitness[:(i+1)])/sum(fitness)

    # Indexes of selected parents
    idxs = np.zeros((2,))
    
    for i in range(2):
        r = float(np.random.uniform(low = 0, high = 1))
        idxs[i] = np.argmin(r < a) - 1
    
    # Selected parents
    parent_a, parent_b = population[idxs.astype(int)]
        
    return parent_a, parent_b

In [17]:
parent_a, parent_b = roulette_wheel(population = population, m = 10, fitness = fitness)
print(parent_a)
print(parent_b)

[0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 0. 1. 1. 0.]
[0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 0. 1. 1. 0.]


In [18]:
def tournament_selection(population, fitness, m, k):
    
    """ Tournament selection. Deterministic and without replacement. """
    
    # Indexes of selected parents
    idxs = np.zeros((2,))
    
    for i in range(2):
        
        # Candidate solutions
        candidates = np.random.choice(range(m), size = k, replace = False)
        candidate_fitness = fitness[candidates]        

        # Selects the best individual in the tournament
        idxs[i] = np.argmin(candidate_fitness)
    
    # Selected parents
    parent_a, parent_b = population[idxs.astype(int)]
        
    return parent_a, parent_b

In [19]:
parent_a, parent_b = tournament_selection(population = population, fitness = fitness, m = 10, k = 10)
print(parent_a)
print(parent_b)

[0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.
 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1.]
[1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1.
 1. 0. 0. 0.]
